# Breakout

In [ ]:
!pip install ale-py==0.7.4
!pip install stable-baselines3[extra]
!pip install torch torchvision torchaudio
!pip install pyglet

In [ ]:
import gym
import os                                                        # System (to save models and logs)
from stable_baselines3 import A2C                                # Import the algorithm we'll use
from stable_baselines3.common.env_util import make_atari_env     # Create multiple env
from stable_baselines3.common.vec_env import VecFrameStack       # Train on them at the same time
from stable_baselines3.common.evaluation import evaluate_policy  # Test the model

## Execute 5 rounds of the game with random actions

In [ ]:
env = gym.make('Breakout-v0')
rounds = 5
for round in range(1, rounds+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        action = env.action_space.sample()
        n_state, reward, done, infos = env.step(action)
        score += reward
    print('Round: {}, Score: {}'.format(round, score))
env.close()

## Use vectors to train the model with multiple environments at the same time

In [ ]:
env = make_atari_env('Breakout-v0', n_envs=4, seed=0)
env = VecFrameStack(env, n_stack=4)

## Declare the model and directory to save ths logs

In [ ]:
a2c_path = os.path.join('Training', 'Saved Models', 'A2C_Breakout-4M')

In [ ]:
log_path = os.path.join('Training', 'Logs')
model = A2C('CnnPolicy', env, verbose = 1, tensorboard_log=log_path)

## Model's learning

In [ ]:
model.learn(total_timesteps=4000000)

## Saving the model

In [ ]:
model.save(a2c_path)
#del model

## Loading a model

In [ ]:
model = A2C.load(os.path.join('Training', 'Saved Models', 'A2C_Breakout-4M'), env)

## Test the model

#### Evaluate on 10 games

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)
# Output with 100 000 timesteps : (7.5, 2.247220505424423)
# Output with 500 000 timesteps : (10.9, 3.2388269481403293)
# Output with 800 000 timesteps : (14.2, 4.833218389437829)
# Output with 1 000 000 timesteps : (16.7, 5.883026432033091)
# Output with 2 000 000 timesteps : (20.3, 9.17660067781093)
# Output with 3 000 000 timesteps : (17.9, 7.542545989253232)
# Output with 4 000 000 timesteps : (21.2, 6.615134163416491)

#### Evaluate on 50 games

In [ ]:
evaluate_policy(model, env, n_eval_episodes=50, render=True)
# Output with 100 000 timesteps : (7.64, 2.5981531902487967)
# Output with 500 000 timesteps : (10.82, 2.8683793333518497)
# Output with 800 000 timesteps : (16.44, 7.797845856388801)
# Output with 1 000 000 timesteps : (15.74, 5.712477571071942)
# Output with 2 000 000 timesteps : (19.56, 8.549058427686642)
# Output with 3 000 000 timesteps : (20.46, 8.66304796246679)
# Output with 4 000 000 timesteps : (17.88, 6.819501448053223)

#### Evaluate on 100 games

In [ ]:
evaluate_policy(model, env, n_eval_episodes=100, render=True)
# Output with 100 000 timesteps : (7.07, 2.0555048041782826)
# Output with 500 000 timesteps : (10.66, 3.6829879174387745)
# Output with 800 000 timesteps : (16.13, 7.400885082204695)
# Output with 1 000 000 timesteps : (16.44, 6.2214467770768564)
# Output with 2 000 000 timesteps : (17.88, 7.303807226371737)
# Output with 3 000 000 timesteps : (19.99, 8.09875916421769)
# Output with 4 000 000 timesteps : (20.59, 8.037530715337889)

# Make a GIF

In [ ]:
import imageio
import numpy as np

In [ ]:
gif_env = make_atari_env('Breakout-v0', seed=0)
gif_env = VecFrameStack(gif_env, n_stack=4)

In [ ]:
gif_model = A2C.load(a2c_path, gif_env)

In [ ]:
images = []
obs = gif_env.reset()
img = gif_env.render(mode='rgb_array')

for i in range(350):
    images.append(img)
    action, _ = gif_model.predict(obs)
    obs, _, _, _ = gif_env.step(action)
    img = gif_env.render(mode='rgb_array')

imageio.mimsave('breakout11.gif',
                [np.array(img) for i, img in enumerate(images) if i % 2 == 0],
                fps=29)
gif_env.close()